In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

from sklearn.neighbors import NearestNeighbors

from sklearn.manifold import TSNE
import plotly.express as px



In [2]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 17.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.4.60
    Uninstalling opencv-python-4.5.4.60:
      Successfully uninstalled opencv-python-4.5.4.60


In [3]:
from deepface import DeepFace

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [4]:
folder_path = '/kaggle/input/top100actors-top100actresses' # provide your path here

file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
for n, i in enumerate(file_list): print(n, os.path.splitext(os.path.basename(i))[0])

0 clark_gable
1 elizabeth_banks
2 frances_mcdormand
3 daisy_ridley
4 rachel_mcadams
5 orson_welles
6 saoirse_ronan
7 jeremy_irons
8 julia_roberts
9 anthony_hopkins
10 jon_voight
11 natalie_portman
12 jennifer_lawrence
13 cate_blanchett
14 hilary_swank
15 alicia_vikander
16 jack_lemmon
17 liv_ullmann
18 anna_kendrick
19 will_smith
20 james_stewart
21 philip_seymour_hoffman
22 daniel_day-lewis
23 heath_ledger
24 robert_downey_jr.
25 grace_kelly
26 alyson_hannigan
27 sigourney_weaver
28 kim_basinger
29 richard_burton
30 john_wayne
31 marion_cotillard
32 anthony_quinn
33 carey_mulligan
34 michelle_pfeiffer
35 leonardo_dicaprio
36 kirsten_dunst
37 christopher_walken
38 sally_hawkins
39 julianne_moore
40 don_cheadle
41 ben_kingsley
42 demi_moore
43 charles_chaplin
44 talia_shire
45 tom_hanks
46 alan_arkin
47 elizabeth_olsen
48 blake_lively
49 brad_pitt
50 jennifer_jason_leigh
51 colin_firth
52 kirk_douglas
53 brie_larson
54 carrie_fisher
55 kate_mara
56 jack_nicholson
57 tommy_lee_jones
58 c

There are several models available in DeepFace:
- VGG-Face
- FaceNet (128D, 512D)
- OpenFace
- DeepID
- DeepFace
- ArcFace

They are different in performance

In [5]:
deepface_vectors = np.array([np.array(DeepFace.represent(i, model_name='DeepFace', enforce_detection=False)[0]['embedding']) for i in file_list])

VGGFace2_DeepFace_weights_val-0.9034.h5 will be downloaded...


Downloading...
From: https://github.com/swghosh/DeepFace/releases/download/weights-vggface2-2d-aligned/VGGFace2_DeepFace_weights_val-0.9034.h5.zip
To: /root/.deepface/weights/VGGFace2_DeepFace_weights_val-0.9034.h5.zip
100%|██████████| 511M/511M [00:08<00:00, 63.4MB/s] 


In [6]:
knn_deepface = NearestNeighbors(n_neighbors=len(deepface_vectors), n_jobs=-1)

knn_deepface.fit(deepface_vectors)

deepface_distances, deepface_indices = knn_deepface.kneighbors(deepface_vectors)

In [8]:
deepface_df = pd.DataFrame([os.path.splitext(os.path.basename(i))[0] for i in file_list])\
        .rename(columns={0:'name'})
deepface_df = pd.concat([deepface_df, pd.DataFrame(deepface_indices[:,1:])], axis=1)#.rename(columns={0:'vector'})

In [9]:
deepface_df[deepface_df.columns[1:]] = deepface_df[deepface_df.columns[1:]].applymap(lambda i: deepface_df.name.iloc[i])
deepface_df.head()

,name,0,1,2,3,4,5,6,7,8,...,189,190,191,192,193,194,195,196,197,198
0,clark_gable,matthew_mcconaughey,hugh_jackman,jennifer_aniston,don_cheadle,tommy_lee_jones,morgan_freeman,julianne_moore,christopher_walken,ben_kingsley,...,viola_davis,carey_mulligan,m�lanie_laurent,emma_stone,laurence_olivier,elizabeth_banks,shelley_duvall,virginia_cherrill,liv_ullmann,kim_novak
1,elizabeth_banks,carey_mulligan,nicole_kidman,kate_winslet,emma_watson,natalie_portman,jim_carrey,elizabeth_olsen,katheryn_winnick,chlo�_grace_moretz,...,charles_chaplin,kim_novak,buster_keaton,viola_davis,l�a_seydoux,margot_robbie,janet_leigh,sean_connery,ingrid_bergman,bibi_andersson
2,frances_mcdormand,jennifer_lawrence,amy_adams,angelina_jolie,robin_wright,naomi_watts,peter_finch,brie_larson,alicia_vikander,kate_mara,...,james_cagney,bette_davis,jack_lemmon,elizabeth_banks,joaquin_phoenix,shelley_duvall,marlene_dietrich,kim_novak,virginia_cherrill,liv_ullmann
3,daisy_ridley,alicia_vikander,angelina_jolie,brie_larson,benicio_del_toro,jennifer_connelly,colin_firth,amy_adams,saoirse_ronan,jennifer_lawrence,...,elizabeth_banks,emma_stone,charles_laughton,bette_davis,marlene_dietrich,joaquin_phoenix,laurence_olivier,virginia_cherrill,liv_ullmann,kim_novak
4,rachel_mcadams,maggie_gyllenhaal,michael_douglas,hugh_jackman,diane_kruger,ellen_burstyn,daniel_day-lewis,matthew_mcconaughey,faye_dunaway,amy_adams,...,talia_shire,m�lanie_laurent,charles_laughton,marlene_dietrich,emma_stone,elizabeth_banks,laurence_olivier,kim_novak,liv_ullmann,virginia_cherrill


In [45]:
overall_distances_df = pd.concat([pd.DataFrame([os.path.splitext(os.path.basename(i))[0] for i in file_list])\
        .rename(columns={0:'name'}), pd.DataFrame(deepface_distances[:,1:])], axis=1).set_index('name').T

overall_indices_df = pd.concat([pd.DataFrame([os.path.splitext(os.path.basename(i))[0] for i in file_list])\
        .rename(columns={0:'name'}), pd.DataFrame(deepface_indices[:,1:])], axis=1).set_index('name').T

overall_scoring_df = overall_distances_df.copy()

for column in overall_scoring_df.columns:
    overall_scoring_df[column] = overall_scoring_df[column].apply(lambda x: round((x- overall_scoring_df[column].max())/(overall_scoring_df[column].min()-overall_scoring_df[column].max())*100, 1) - 1 if x >1.01 else x).apply(abs)

overall_scoring_df.head(10)

name,clark_gable,elizabeth_banks,frances_mcdormand,daisy_ridley,rachel_mcadams,orson_welles,saoirse_ronan,jeremy_irons,julia_roberts,anthony_hopkins,...,charlize_theron,steve_mcqueen,michael_douglas,rachel_weisz,chlo�_grace_moretz,m�lanie_laurent,meryl_streep,robert_redford,katheryn_winnick,maggie_gyllenhaal
0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
1,96.5,97.3,97.9,98.5,95.7,99.0,98.8,94.2,97.6,89.9,...,97.6,98.6,98.8,98.1,98.6,97.4,97.5,95.2,97.8,94.1
2,91.3,96.4,97.7,98.1,94.5,96.4,95.0,94.0,92.2,89.8,...,97.1,97.0,97.8,97.5,90.9,94.7,97.0,92.2,96.1,92.7
3,90.0,87.3,95.0,97.2,93.2,96.4,94.9,92.0,90.7,89.1,...,96.1,96.6,97.2,96.5,89.4,91.7,95.9,91.1,86.9,91.8
4,88.8,84.8,94.7,96.9,91.8,95.5,94.6,91.0,89.1,87.8,...,93.9,96.4,97.1,95.6,89.3,91.0,95.6,90.6,81.7,90.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,12.4,15.6,11.6,9.1,5.3,8.7,13.2,20.6,12.5,3.2,...,11.5,13.9,10.9,17.6,10.6,6.6,11.9,18.9,10.6,11.1
195,10.6,14.3,10.8,8.6,3.3,3.6,10.9,18.6,9.0,2.1,...,10.1,13.8,8.8,15.6,9.6,5.6,11.8,17.5,10.5,8.3
196,6.4,6.1,7.1,4.1,2.0,2.9,6.0,18.5,8.6,0.2,...,7.4,10.5,5.9,13.6,4.9,0.5,8.4,17.3,6.7,4.9
197,3.0,2.6,1.0,3.2,1.3,0.6,4.5,6.7,3.0,0.5,...,4.4,5.9,3.4,3.8,3.8,0.4,7.5,7.7,1.0,3.5


In [22]:
names_df = deepface_df.set_index('name').T
names_df.head()

In [32]:
hidden_celebrity = 'saoirse_ronan'

In [43]:
def celeb_and_score_query(celebrity, hidden_celebrity=hidden_celebrity, names_df=names_df, score_df=overall_scoring_df):
    celeb_rank = names_df[hidden_celebrity][names_df[hidden_celebrity] == celebrity].index
    celeb_score = overall_scoring_df[hidden_celebrity].loc[celeb_rank].iloc[0]
    return celebrity, celeb_score
    

In [44]:
celeb_and_score_query('hugh_jackman')

('hugh_jackman', 44.9)

In [17]:

tsne = TSNE(n_components=3, random_state=0, perplexity=5)
projections = tsne.fit_transform(deepface_vectors, )

plotting_df = pd.concat([deepface_df['name'], pd.DataFrame(projections)], axis=1)

fig = px.scatter_3d(
    plotting_df, x=0, y=1, z=2, hover_data='name'
)
fig.update_traces(marker_size=4)
fig.show()